In [1]:
#импорт библиотек
import pandas as pd
import pickle as pkl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold
from sklearn.metrics import mean_squared_log_error as MSLE
import pymorphy2 # Морфологический анализатор.
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
import umap
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.cluster import DBSCAN
# Импортируем библиотеки Word2Vec
from gensim.models.word2vec import Word2Ve
import numpy as np # Вектора.

C:\Users\gavrilov_ej\AppData\Local\Continuum\anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


ImportError: cannot import name 'Word2Ve'

In [ ]:
#Начальные константы
%matplotlib inline

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12,5)
morph=pymorphy2.MorphAnalyzer() # Создает объект морфоанализатора и загружет словарь.

posConv={'ADJF':'_ADJ','NOUN':'_NOUN','VERB':'_VERB'}
#meaningfullPoSes=['ADJF', 'NOUN', 'VERB']
meaningfullPoSes=['NOUN']
meaningfullAdj=['ADJF']
#model_w2v = KeyedVectors.load_word2vec_format('data/rusvec.vec.gz')

# Функции

In [ ]:
#Для удаления полей, которые я не буду использовать
def DeleteUnnesField(DF):
    new_DF = DF.drop(columns=['contacts_visible', 'date_created', 'description',\
                              'fields', 'images', 'location', 'subway'], axis = 1)
    return new_DF

In [2]:
def GetNameString(NameString):
    
    name   = str(NameString);
    words      = name.split();
    resultlist = []

    for w in words:
        
        if len(w) < 2:
            continue
        
        wordform = morph.parse(w)[0]
        resultlist.append(wordform.normal_form)
    
    return resultlist
    

In [3]:
#Для определения субкласса в дополнение к категории и субкатегории по 
#первому существительному и прилагательному
def GetVecByName(col_name):
    
    Vec        = np.zeros(shape=(100))  
    count      = 0
    
    for word in set(col_name):
        try:
            Vec+= modelw2v[word]
            count+=1
        except:
            count=count
            
    return np.dot(Vec, np.ones(100))
    

In [4]:
#не используется
#Для определения субкласса в дополнение к категории и субкатегории по 
#первому существительному и прилагательному
def GetClassByName(col_name):
    
    name   = str(col_name);
    words      = name.split();
    resultlist = []

    for w in words:
        wordform = morph.parse(w)[0]
        if wordform.tag.POS in meaningfullPoSes:
            resultlist.append(wordform.normal_form+posConv[wordform.tag.POS])
            #resultlist.append(wordform.normal_form)
            break
    
    for w in words:
        wordform = morph.parse(w)[0]
        if wordform.tag.POS in meaningfullAdj:
            resultlist.append(wordform.normal_form+posConv[wordform.tag.POS])
            #resultlist.append(wordform.normal_form)
            break

    Vec        = np.zeros(shape=(600))  
    count      = 0
    
    for word in set(resultlist):
        try:
            Vec+= model_w2v[word]
            count+=1
        except:
            #print(word)
            count=count
    
    if count > 0:
        Vec = Vec/count

    return np.dot(Vec, np.ones(600))


In [5]:
#Не используется. Изначально хотел по нормализованным словам получить класс,
#но различие тестовой и тренировочной выборки не позволило
def GetSubclasses(DF):
    category = set(DF.category)
    subcateg = set(DF.subcategory)
    resultDF = DF[DF.category==-1]

    for i in category:
        for j in subcateg:
            df = DF[(DF.category==i) & (DF.subcategory==j)]
        
            if len(df)==0:
                 continue
                    
            WordSet = set(df.norm_name)    
            
            #составление словаря
            WordDict = {}
            count = 0
            for item in WordSet:
                WordDict.update({item: count})
                count+=1

            #Получение подкласса
            df.loc[:,'subclass'] = df.loc[:,'norm_name'].apply(lambda x: WordDict.get(x))
   
            resultDF=pd.concat([resultDF, df])
            
            #break
        #break
    return resultDF

In [20]:
#Читаем тренировочную выборку 
with open('data/train_hack.pckl','rb') as f:
    data_new = pkl.load(f)

In [21]:
#Читаем тестовую выборку
with open('data/test_hack.pckl','rb') as f:
    data_test = pkl.load(f)

In [22]:
#удаляем столбцы
train_norm = DeleteUnnesField(data_new)
test_norm = DeleteUnnesField(data_test)

In [23]:
#Освобождаю память
data_new = 1
data_test = 1


In [26]:
#Получение представления наименования в виде числа
train_norm['det'] = train_norm['name'].map(GetClassByName)
test_norm['det'] = test_norm['name'].map(GetClassByName)

In [29]:
#Получение "нормальных" имен
train_norm['norm_name'] = train_norm['name'].map(GetNameString)
test_norm['norm_name'] = test_norm['name'].map(GetNameString)

In [31]:
Sentences = train_norm['norm_name'].tolist() + test_norm['norm_name'].tolist()

In [32]:
#Обучение модели для имен
modelw2v = Word2Vec(sentences=Sentences,seed=127)

In [34]:
#Получение "субкласса" товара
train_norm['det'] = train_norm['norm_name'].map(GetVecByName)
test_norm['det'] = test_norm['norm_name'].map(GetVecByName)

C:\Users\gavrilov_ej\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.


In [36]:
#трейн
X = train_norm.drop(columns=['id','name','price','norm_name'])
y = train_norm.price/100

In [ ]:
#фит
x_train, x_test, y_train, y_test = train_test_split(X,y,test_size=0.3)
y_test = np.array(y_test)

#model = RandomForestRegressor()

model = Pipeline([
    ('onehot', OneHotEncoder(categorical_features=[2,6])),
    ('reg', RandomForestRegressor(random_state=127, n_estimators=20))
])

model.fit(x_train, y_train)

In [ ]:
yhat = model.predict(X)
MSLE(y, yhat)

In [77]:
# Задаем значения параметров, которые хотим проверить
# Теперь с распределениями
param_grid = {
    'reg__n_estimators': range(10,40,5),
    'reg__criterion': ['mse','mae'],
    'reg__max_depth': range(2, 8),
    'reg__mean_samples_leaf':range(1,100)
}

# Задаем схему кросс-валидации
cv = StratifiedKFold(n_splits=5, random_state=127, shuffle=True)

In [173]:
# Рассмотрим 20 случайных комбинаций
#random_searcher = Ran(model, param_grid, n_iter=10, 
#                                     random_state=127,
#                                      
#                                     n_jobs=-1, cv=cv, 
#                                     verbose=2,error_score=0)

#random_searcher.fit(X, y)

In [26]:
X_test = test_norn.drop(columns=['id','name','norm_name'])

In [27]:
y_test_hat = model.predict(X_test)
#MSLE(y_test, yhat)

In [182]:
samplesub = pd.read_csv('data/submit_Sample.csv')

In [29]:
test_norn['price']=y_test_hat

In [31]:
#Хоть какой-то результат для kaggle
test_norn.to_csv('predict.csv', index=None,columns=['id','price'])

In [199]:
on_this = train_norm[(train_norm.category==22) & (train_norm.subcategory==2204)]

In [217]:
curmodel = Word2Vec(sentences=on_this.norm_name.tolist())

In [244]:
on_this['det'] = on_this['norm_name'].map(lambda x: GetVecByName1(x, curmodel))

C:\Users\gavrilov_ej\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.
C:\Users\gavrilov_ej\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [245]:
X = on_this.drop(columns=['id','name','price','norm_name','predict'])
y = on_this.price

In [246]:
#фит
x_train, x_test, y_train, y_test = train_test_split(X,y,test_size=0.3)
y_test = np.array(y_test)

#model = RandomForestRegressor()

model2 = Pipeline([
    ('onehot', OneHotEncoder(categorical_features=[2,6])),
    ('reg', RandomForestRegressor(random_state=127, n_estimators=20))
])

model2.fit(x_train, y_train)

Pipeline(memory=None,
     steps=[('onehot', OneHotEncoder(categorical_features=[2, 6], dtype=<class 'numpy.float64'>,
       handle_unknown='error', n_values='auto', sparse=True)), ('reg', RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           mi...stimators=20, n_jobs=1,
           oob_score=False, random_state=127, verbose=0, warm_start=False))])

In [247]:
yhat = model.predict(x_test)
MSLE(y_test, yhat)

1.513834408724913